<a href="https://colab.research.google.com/github/jesicagimenez94/DataAnalyst/blob/main/Modelo_Estrella.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import requests
import pandas as pd

# Conexión a la base de datos SQLite
def connect_to_sqlite(db_name="rick_and_morty.db"):
    conn = sqlite3.connect(db_name)
    return conn

# Extraemos los datos de la API de Rick and Morty
def extract_data_from_api():
    base_url = "https://rickandmortyapi.com/api/character"
    characters = []

    while base_url:
        response = requests.get(base_url)
        if response.status_code == 200:
            data = response.json()
            characters.extend(data.get("results", []))
            base_url = data.get("info", {}).get("next")  # Cambiar al siguiente paginado
        else:
            print(f"Error en la solicitud: {response.status_code}")
            break

    print(f"Se han extraído {len(characters)} personajes.")
    return characters

# Limpiamos y organizamos los datos
def transform_data(characters):
    df = pd.DataFrame(characters)

    # Dejamos solo las columnas importantes
    relevant_columns = ["id", "name", "status", "species", "gender", "origin", "location"]
    df = df[relevant_columns]

    # Normalizamos las columnas de tipo diccionario (origin y location)
    df["origin_name"] = df["origin"].apply(lambda x: x["name"] if isinstance(x, dict) else "")
    df["location_name"] = df["location"].apply(lambda x: x["name"] if isinstance(x, dict) else "")
    df.drop(columns=["origin", "location"], inplace=True)

    print(f"Datos transformados, total de registros limpios: {df.shape[0]}")
    return df

# Creamos las tablas de dimensiones (en este caso, extraemos las categorías únicas)
def create_dimension_tables(df, conn):
    dim_status = pd.DataFrame(df['status'].unique(), columns=['status_name'])
    dim_gender = pd.DataFrame(df['gender'].unique(), columns=['gender_name'])
    dim_species = pd.DataFrame(df['species'].unique(), columns=['species_name'])
    dim_origin = pd.DataFrame(df['origin_name'].unique(), columns=['origin_name'])
    dim_location = pd.DataFrame(df['location_name'].unique(), columns=['location_name'])

    # Insertamos las dimensiones en la base de datos
    dim_status.to_sql('DimStatus', conn, if_exists='replace', index=False)
    dim_gender.to_sql('DimGender', conn, if_exists='replace', index=False)
    dim_species.to_sql('DimSpecies', conn, if_exists='replace', index=False)
    dim_origin.to_sql('DimOrigin', conn, if_exists='replace', index=False)
    dim_location.to_sql('DimLocation', conn, if_exists='replace', index=False)

    print("Tablas de dimensiones cargadas en la base de datos.")
    return dim_status, dim_gender, dim_species, dim_origin, dim_location

# Creamos la tabla de hechos
def create_fact_table(df, dim_status, dim_gender, dim_species, dim_origin, dim_location, conn):
    # Mapeamos las claves de las dimensiones
    df['status_id'] = df['status'].apply(lambda x: dim_status[dim_status['status_name'] == x].index[0] + 1)
    df['gender_id'] = df['gender'].apply(lambda x: dim_gender[dim_gender['gender_name'] == x].index[0] + 1)
    df['species_id'] = df['species'].apply(lambda x: dim_species[dim_species['species_name'] == x].index[0] + 1)
    df['origin_id'] = df['origin_name'].apply(lambda x: dim_origin[dim_origin['origin_name'] == x].index[0] + 1)
    df['location_id'] = df['location_name'].apply(lambda x: dim_location[dim_location['location_name'] == x].index[0] + 1)

    fact_table = df[['id', 'status_id', 'gender_id', 'species_id', 'origin_id', 'location_id']]
    fact_table.rename(columns={'id': 'character_id'}, inplace=True)

    # Insertamos la tabla de hechos en la base de datos
    fact_table.to_sql('FactCharacters', conn, if_exists='replace', index=False)

    print("Tabla de hechos cargada en la base de datos.")
    return fact_table

# Realizamos consultas SQL para mostrar datos
def run_sql_queries(conn):
    # Mostramos los primeros 5 registros de cada tabla de dimensión
    print("\nPrimeros 5 registros de la tabla DimStatus:")
    dim_status_query = "SELECT * FROM DimStatus LIMIT 5;"
    print(pd.read_sql_query(dim_status_query, conn))

    print("\nPrimeros 5 registros de la tabla DimGender:")
    dim_gender_query = "SELECT * FROM DimGender LIMIT 5;"
    print(pd.read_sql_query(dim_gender_query, conn))

    print("\nPrimeros 5 registros de la tabla DimSpecies:")
    dim_species_query = "SELECT * FROM DimSpecies LIMIT 5;"
    print(pd.read_sql_query(dim_species_query, conn))

    print("\nPrimeros 5 registros de la tabla DimOrigin:")
    dim_origin_query = "SELECT * FROM DimOrigin LIMIT 5;"
    print(pd.read_sql_query(dim_origin_query, conn))

    print("\nPrimeros 5 registros de la tabla DimLocation:")
    dim_location_query = "SELECT * FROM DimLocation LIMIT 5;"
    print(pd.read_sql_query(dim_location_query, conn))

    print("\nPrimeros 5 registros de la tabla FactCharacters:")
    fact_characters_query = "SELECT * FROM FactCharacters LIMIT 5;"
    print(pd.read_sql_query(fact_characters_query, conn))

    # Otras Consultas
    print("\nCantidad de personajes por estado (DimStatus):")
    state_count_query = """
    SELECT ds.status_name, COUNT(fc.character_id) AS character_count
    FROM FactCharacters fc
    JOIN DimStatus ds ON fc.status_id = ds.rowid
    GROUP BY ds.status_name;
    """
    print(pd.read_sql_query(state_count_query, conn))

    print("\nCantidad de personajes por género (DimGender):")
    gender_count_query = """
    SELECT dg.gender_name, COUNT(fc.character_id) AS character_count
    FROM FactCharacters fc
    JOIN DimGender dg ON fc.gender_id = dg.rowid
    GROUP BY dg.gender_name;
    """
    print(pd.read_sql_query(gender_count_query, conn))

# Ejecutamos el proceso ETL, cargamos datos en SQLite y mostramos los resultados
def run_etl_and_load_to_sqlite():
    # Conectamos a la base de datos SQLite
    conn = connect_to_sqlite()

    # Extracción
    characters = extract_data_from_api()

    # Transformación
    transformed_data = transform_data(characters)

    # Creamos las tablas de dimensiones
    dim_status, dim_gender, dim_species, dim_origin, dim_location = create_dimension_tables(transformed_data, conn)

    # Creamos la tabla de hechos
    create_fact_table(transformed_data, dim_status, dim_gender, dim_species, dim_origin, dim_location, conn)

    # Ejecutamos consultas SQL para mostrar los resultados
    run_sql_queries(conn)

    # Cerramos la conexión a la base de datos
    conn.close()

# Ejecutamos el proceso ETL y cargar los datos en SQLite
run_etl_and_load_to_sqlite()


Se han extraído 826 personajes.
Datos transformados, total de registros limpios: 826
Tablas de dimensiones cargadas en la base de datos.


<ipython-input-1-838ad7457a5d>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fact_table.rename(columns={'id': 'character_id'}, inplace=True)


Tabla de hechos cargada en la base de datos.

Primeros 5 registros de la tabla DimStatus:
  status_name
0       Alive
1     unknown
2        Dead

Primeros 5 registros de la tabla DimGender:
  gender_name
0        Male
1      Female
2     unknown
3  Genderless

Primeros 5 registros de la tabla DimSpecies:
    species_name
0          Human
1          Alien
2       Humanoid
3        unknown
4  Poopybutthole

Primeros 5 registros de la tabla DimOrigin:
                     origin_name
0                  Earth (C-137)
1                        unknown
2  Earth (Replacement Dimension)
3                       Abadango
4               Signus 5 Expanse

Primeros 5 registros de la tabla DimLocation:
                   location_name
0               Citadel of Ricks
1  Earth (Replacement Dimension)
2                       Abadango
3     Testicle Monster Dimension
4              Worldender's lair

Primeros 5 registros de la tabla FactCharacters:
   character_id  status_id  gender_id  species_id  or